In [1]:
import matplotlib 

matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': 'serif',
    'text.usetex': True,
    'pgf.rcfonts': False,
})

import matplotlib.pyplot as plt

SMALL_SIZE = 8
MEDIUM_SIZE = 10
BIGGER_SIZE = 12

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)    # fontsize of the axes title
plt.rc('axes', labelsize=SMALL_SIZE)     # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [2]:
#import necessary packages
import subprocess
import numpy as np
import os.path
import os
import sys
import glob as glob
#import matplotlib.pyplot as plt

class cd:
    """Context manager for changing the current working directory"""
    def __init__(self, newPath):
        self.newPath = os.path.expanduser(newPath)

    def __enter__(self):
        self.savedPath = os.getcwd()
        os.chdir(self.newPath)

    def __exit__(self, etype, value, traceback):
        os.chdir(self.savedPath)

In [3]:
Ns      = np.arange(2, 6, 1)
Ns

array([2, 3, 4, 5])

In [9]:
#run the program using o2 optimization flag

subprocess.run(["gfortran", "Ex10-Segalini-CODE.f90", "-llapack", "-o", "./run.exe", "-O2"])

# path = "./data/"

#vector containing execution times 
times    = np.zeros((len(Ns), 2))

for i in range(len(Ns)):
    # lambda is set equal to 0.5
    output = subprocess.run(['./run.exe'], stdout=subprocess.PIPE, 
                            input="{}\n".format(Ns[i]), encoding="ascii")
    print('N = ', Ns[i])    


N =  2
N =  3
N =  4
N =  5


In [4]:
# np.savetxt("./times.dat", times, fmt='%f')
#data = np.loadtxt(glob.glob('*.dat'))

data = np.zeros((len(Ns), 301, 5))

filenames = sorted(glob.glob('*.dat'))

for i, file in enumerate(filenames):
    print("FILE = ", file)
    with open(file) as f:
        for j, line in enumerate(f):
            output = line.split()   
            #print(output)
            data[i, j, 0] = output[0]
            #print("lambda=", data[i, j, 0])
            data[i, j, 1] = output[1]
            #print("e0=", data[i, j, 1])
            data[i, j, 2] = output[2]
            #print("fsize=", data[i, j, 2])
            data[i, j, 3] = output[3]
            #print("n_it=", data[i, j, 3])
            data[i, j, 4] = output[4]
            #print("exec_time=", data[i, j, 4])


FILE =  N002.dat
FILE =  N003.dat
FILE =  N004.dat
FILE =  N005.dat


In [12]:
fig, ax = plt.subplots(1, 2, figsize=(6.68, 2.2))
#fig, ax = plt.subplots(1, 1, figsize=(14, 6))

l = np.linspace(0,3,301)
mf = -1-(l[0:201])**2/4
mf2 = -abs(l[201:])

tot_mf = np.concatenate((mf, mf2))

ax[0].plot(data[i, :, 0], tot_mf, c='r', label="MF solution")

col = ['springgreen', 'coral', 'plum', 'cornflowerblue']
for i in range(len(Ns)):
    ax[0].plot(data[i, :, 0], data[i, :, 1], c = col[i], label="N = "+str(Ns[i]))

ax[0].set_xlabel("External interaction strength $\lambda$")
ax[0].set_ylabel("$E_0/N$")
ax[0].legend(loc='lower left')

ax[1].set_xscale('log')
ax[1].plot(data[i, :, 0], tot_mf, c='r', label="MF solution")

col = ['springgreen', 'coral', 'plum', 'cornflowerblue']
for i in range(len(Ns)):
    ax[1].plot(data[i, :, 0], data[i, :, 1], c = col[i], label="N = "+str(Ns[i]))

ax[1].set_xlabel("Logscale of external interaction strength $\lambda$")
ax[1].set_ylabel("$E_0/N$")
ax[1].legend(loc='lower left')

plt.tight_layout()
plt.show()
plt.savefig('gs.pgf')

D:\Anaconda\envs\tf\lib\site-packages\ipykernel_launcher.py:32: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.


In [15]:
exec_time = []
err_t = []

for i in range(len(Ns)):
    exec_time.append(np.mean(data[i, :, 4]))
    err_t.append(np.std(data[i, :, 4]))

fig, ax = plt.subplots(1, 1, figsize=(4, 3))
#fig, ax = plt.subplots(1, 1, figsize=(10, 6))

ax.set_xlabel('Number of particles $N$')
ax.set_ylabel('Execution time [s] in logscale')

#ax.set_title('RSRG iteration average execution time')
ax.set_yscale('log')

ax.errorbar(Ns, exec_time, yerr=err_t, fmt = '--x', ecolor = 'r', c = 'orange', label="$t \pm \sigma_t$")
ax.legend(loc='upper left')

plt.tight_layout()
plt.show()
plt.savefig('times.pgf')

D:\Anaconda\envs\tf\lib\site-packages\ipykernel_launcher.py:21: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.


In [8]:
fig, ax = plt.subplots(1, 2, figsize=(6.68, 2.2))
#fig, ax = plt.subplots(1, 2, figsize=(14, 6))

col = ['springgreen', 'coral', 'plum', 'cornflowerblue']
for i in range(len(Ns)):
    ax[0].plot(data[i, :, 0], data[i, :, 3], c = col[i], label="N = "+str(Ns[i]))

ax[0].set_xlabel("External interaction strength $\lambda$")
ax[0].set_ylabel("Number of iterations")
ax[0].legend(loc='lower left')

avg_n_it = []
err_n_it = []

for i in range(len(Ns)):
    avg_n_it.append(np.mean(data[i, :, 3]))
    err_n_it.append(np.std(data[i, :, 3]))

ax[1].set_xlabel("Number of particles $N$")
ax[1].set_ylabel("Average number of iterations")

#ax[1].set_yscale('log')

ax[1].errorbar(Ns, avg_n_it, yerr=err_n_it, fmt = '--x', ecolor = 'r', c = 'orange', label="$n_{it} \pm \sigma_{n_{it}}$")
ax[1].legend(loc='lower right')


plt.tight_layout()
plt.show()
plt.savefig('iterations.pgf')

D:\Anaconda\envs\tf\lib\site-packages\ipykernel_launcher.py:29: UserWarning: Matplotlib is currently using pgf, which is a non-GUI backend, so cannot show the figure.
